# Metrics 

> Metric to quantify circadian disruption 

In [ ]:
#| default_exp metrics

In [ ]:
#| hide 
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
from fastcore.basics import *

In [ ]:
#| export
import numpy as np
from circadian.models import Hannay19
from circadian.utils import phase_ic_guess
from circadian.readers import WearableData

In [ ]:
#| export
#| hide
def esri(awObj: WearableData, 
                gamma: float = 0.0,
                multiplier: float = 1.0,
                num_days: float = 4.5):

    spmodel = Hannay19({'K': 0.0, 'gamma': gamma})
    psi0 = phase_ic_guess(awObj.time_total[0])

    idx = awObj.time_total < awObj.time_total[0]+24*num_days
    sol = spmodel.integrate(awObj.time_total[idx], 
                            np.array([0.10, psi0, 0.0]),
                            multiplier*awObj.steps[idx],) 
    return sol.time, sol.states

In [ ]:
#| export
#| hide
def esri_trajectory(awObj: WearableData,
                    gamma: float = 0.0,
                    multiplier: float = 1.0,
                    num_days: float = 4.5,
                    ):
    spmodel = Hannay19({'K': 0.0, 'gamma': gamma})
    compactness_trajectory = []
    time_trajectory = []
    timestamps = []
    timeStart = awObj.time_total[0]
    timestampStart = awObj.timestamp[0]
    while timeStart < awObj.time_total[-1] - 24*num_days:
        try:
            psi0 = phase_ic_guess(timeStart)
            idxStart = (awObj.time_total > timeStart)
            tsFilter = awObj.time_total[idxStart]
            idx = tsFilter < np.array(tsFilter[0])+24*num_days
            stepsFilter = awObj.steps[idxStart]
            trajectory = spmodel.integrate(
                tsFilter[idx], np.array([0.10, psi0, 0.0], 
                multiplier*stepsFilter[idx]))
            sol = trajectory.states
            if sol[0, -1] > 0.0:
                compactness_trajectory.append(sol[0, -1])
                time_trajectory.append(timeStart)
                timestamps.append(
                    3600.0*(timeStart - awObj.time_total[0]) + timestampStart)
        except:
            print("Error in trajectory")
        timeStart += 1.0
    return time_trajectory, timestamps, compactness_trajectory

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()